In [ ]:
import pandas as pd
import math
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 500)

In [ ]:
#filePath = 'data/arc2yeardata.xlsx'
filePath = '/Volumes/OM/ECO/M&V/Holiday Shutdowns/Memorial Day 2018/Archive/Campus Data 052918.csv'
#data = pd.read_excel(filePath, header=0, index_col=0, parse_date=True)
data = pd.read_csv(filePath, header=0, index_col=0, parse_dates=True)
print(data.columns)

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
toolPath = r'/Users/decolvin/Box Sync/UCD_ECO_coding/mypy'
sys.path.append(toolPath)

import mnv
import matplotlib.pyplot as plt
print("Success!")

In [ ]:
data.tail()

In [ ]:
inputDict = {'IQRmult' : 3.0,
                 'IQR' : 'y',
                 'ceiling' : 1000000,
                 'floor' : 0,
                 'resampleRate' : 'D',
                 'verbosity' : 3,
                 'sliceType' : 'ranges', #half, middate, ranges
                 'midDate' : '2018-01-01', #only needed with sliceType : 'middate'
                 'dateRanges' : ['2017-04-01','2018-04-01','2018-05-26','2018-05-28'], #only needed with sliceType : 'ranges'
                 'OATname' : None,
                 'OATsource' : 'file'} #only needed with sliceType : 'ranges'}
data.rename(columns={"chw_tonh":"response"},inplace=True)

dc = mnv.data_handler(data["response"], inputDict)
dc.default_clean()

# Run this incase the y-range is FUBAR
#dc.outlier_plot(dc.modifiedData[dc.com]) 

fig = plt.figure(figsize=(18,5))
ax = fig.add_subplot(111)
ax.plot(dc.pre[dc.com],color='b')
ax.plot(dc.post[dc.com],color='g')
plt.show()

In [ ]:
### Correlation
dc.pre.plot.scatter(x='CDH', y='response',figsize=(18,2), color='blue')
dc.pre.plot.scatter(x='HDH', y='response',figsize=(18,2), color='orange')
plt.show()
print "Response vs CDH = %.2f" %dc.pre.response.corr(dc.pre.CDH)
print "Response vs HDH = %.2f" %dc.pre.response.corr(dc.pre.HDH)
print "Response vs CDH2 = %.2f" %dc.pre.response.corr(dc.pre.CDH2)
print "Response vs HDH2 = %.2f" %dc.pre.response.corr(dc.pre.HDH2)

In [ ]:
dc.pre.columns

In [ ]:
allmod = mnv.many_ols(dc.pre, dc.post, inputParams=['', 'C(month)','C(weekend)'])

allmod.run_all_linear()
print(allmod.statsPool[0:3])
#allmod.plot_pool(2)

topParams = allmod.statsPool.iloc[0]['params']
print
print "Best Paramaters = %s" %topParams

In [ ]:
params = topParams
mod = mnv.ols_model(dc.pre, dc.post, params)

plt.figure()
mod.train.response.plot(figsize=(18,5))
mod.trainCalc.plot(figsize=(18,5))

plt.figure()
mod.model_plot()

plt.show()
mod.Fit.summary()

In [ ]:
rates = {'chw':0.047, 'steam':6, 'elec':0.065} #tonh,klb,and kWh prices for 2017
commodity = 'chw'

mod.savings_plot(yaxis='dollars', rate=rates[commodity])
print "Total Savings over Post Period = $%.0f" %(mod.postCumsum * rates[commodity])
print "R-Squared = %.2f" %mod.Fit.rsquared_adj
print "CVRMSE = %.2f" %(math.sqrt(mod.Fit.mse_resid) / dc.pre.response.mean())